# 9. Pythonic Object
- Python data모델은 사용자가 정의한 자료형도 내장 자료형과 같이 자연스럽게 동작할 수 있음
    - Duck Typing덕분에 특정 자료형을 상속하지 않고도 가능함
    - Object에 필요한 method만을 구현하면 기대한 대로 동작함
- What we are going to do in this chapter?
    - 실제 Python object와 동일하게 동작하는 user-defined class 만들기
- In this chapter,
    - repr(), bytes()등 object를 다른 방식으로 표현하는 내장 함수의 지원
    - class method로 대안 생성자 구현
    - format() 내장 함수와 str.format() method에서 사용하는 포맷 언어 확장
    - 읽기 전용 접근만 허용하는 속성 제공
    - 집합 및 dict key로 사용할 수 있도록 object를 hashable하게 만들기
    - __slots__ 를 사용하여 메모리 절약하기
    - @staticmethod, @classmathod
    - Python에서 비공개 및 보호된 속성: 사용법, 관례, 한계

## 9.1 Object 표현
- repr(): object를 개발자가 보고자 하는 형태로 표현한 문자열로 return
- str(): object를 사용자가 보고자 하는 형태로 표현한 문자열로 return
- repr(), str() method를 지원하려면 __repr()__, __str()__를 구현해야 함
- __bytes__(), __format__()
    - __bytes__(): __str__()와 비슷하지만, bytes() method에 의해 호출되어 object를 byte sequence로 표현함
    - __format__(): 내장함수 format()과 str.format()를 둘다 사용. 특별 format code를 사용해서 object를 표현하는 문자열 반환

## 9.2 Vector Class의 부활

In [42]:
from array import array
import math

class Vector2d_v0:
    typecode = 'd' #Vector2d와 bytes간의 변환에 사용하는 class 속성
    
    def __init__(self, x, y):
        self.x = float(x)
        self.y = float(y)
    
    def __iter__(self):
        return (i for i in (self.x, self.y))
    
    def __repr__(self):
        class_name = type(self).__name__
        return '{}({!r}, {!r})'.format(class_name, *self)
    
    def __str__(self):
        return str(tuple(self))
    
    # bytes를 생성하기 위해 typecode를 bytes로 표현 & 객체를 반복해서 생성한 배열에서 변환된 bytes와 연결
    def __bytes__(self):
        return bytes([ord(self.typecode)]) + bytes(array(self.typecode, self)) 
    
    def __eq__(self, other):
        return tuple(self) == tuple(other)
    
    def __abs__(self):
        return math.hypot(self.x, self.y)
    
    def __bool__(self):
        return bool(abs(self))
    
    @classmethod
    def frombytes(cls, octets):
        typecode = chr(octets[0])
        memv = memoryview(octets[1:]).cast(typecode)
        return cls(*memv)

In [43]:
v1 = Vector2d_v0(3,4)
print('v1.x, v1.y: ', v1.x, v1.y)
x, y = v1 
print('x, y: ', x, y)
print('v1: ', v1)
v1_clone = eval(repr(v1))
print('v1 == v1_clone: ', v1 == v1_clone)
print('v1 is v1_clone: ', v1 is v1_clone)
octets = bytes(v1)
print('bytes(v1): ', octets)
print('abs(v1): ', abs(v1))
print('bool(v1): {}, bool(Vector2d_v0(0, 0)): {}'.format( bool(v1), bool(Vector2d_v0(0, 0))))



v1.x, v1.y:  3.0 4.0
x, y:  3.0 4.0
v1:  (3.0, 4.0)
v1 == v1_clone:  True
v1 is v1_clone:  False
bytes(v1):  b'd\x00\x00\x00\x00\x00\x00\x08@\x00\x00\x00\x00\x00\x00\x10@'
abs(v1):  5.0
bool(v1): True, bool(Vector2d_v0(0, 0)): False


## 9.4 @classmethod와 @staticmethod
- @Classmethod
    - object가 아닌 class에 연산을 수행하는 method
    - class자체를 첫 번째 parameter로 받게 만듦
    - 아래 예제에서 _randomize()_는 __init__()안의 instance variable을 전혀 사용하지 않음. 즉, instance method없이 바로 class에 적용 가능    

In [44]:
# Before
import random
class Randomize1:
    RANDOM_CHOICE = 'abcdefg'
    
    def __init__(self, char_num):
        self.char_num = char_num
    
    def _randomize(self, random_chars=3):
        return ''.join(random.choice(self.RANDOM_CHOICE) for _ in range(random_chars))

    
class Randomize2:
    RANDOM_CHOICE = 'abcdefg'
    
    def __init__(self, char_num):
        self.char_num = char_num
    
    @classmethod
    def _randomize(cls, random_chars=3):
        return ''.join(random.choice(cls.RANDOM_CHOICE) for _ in range(random_chars))

ran1 = Randomize1(5)
print(ran1._randomize())
print(Randomize2._randomize())

cca
egd


- @classmethod
    - 또 다른 예제
    - https://stackoverflow.com/questions/12179271/meaning-of-classmethod-and-staticmethod-for-beginner

In [45]:
class Date(object):
    
    def __init__(self, day=0, month=0, year=0):
        self.day = day
        self.month = month
        self.year = year

    # input이 '10-09-2012' 형식으로 들어올 경우, 이 text를 '-'로 parsing해서 각각 day, month, year에 넣은 Date object 생성
    @classmethod
    def from_string(cls, date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        date1 = cls(day, month, year)
        return date1
    
    # classmethod나 object method처럼 첫번째 param에 cls나 self가 들어가지 않음
    @staticmethod
    def is_date_valid(date_as_string):
        day, month, year = map(int, date_as_string.split('-'))
        return day <= 31 and month <=12 and year  <= 3000
    
date2 = Date.from_string('11-09-2012')
is_date = Date.is_date_valid('11-09-2012')
print('{}/{}/{} is a date : {}'.format(date2.year, date2.month, date2.day, is_date))

2012/9/11 is a date : True


In [46]:
class Demo:
    @classmethod
    def klassmeth(*args):
        return args
    @staticmethod
    def statmeth(*args):
        return args
    
print(Demo.klassmeth())
print(Demo.klassmeth('spam'))
print(Demo.statmeth())
print(Demo.statmeth('spam'))

(<class '__main__.Demo'>,)
(<class '__main__.Demo'>, 'spam')
()
('spam',)


## 9.5 Formatted print
- format()내장 함수와 str.format() method는 실제 format 작업을 __format__(format_spec) method에 위임함
    - format_spec: 포맷 명시자(format specifier)
        - format(my_obj, format_spec)의 2번째 인수
        - str.format()에 사용된 format string 안에 {}로 구분한 대체 필드 안에서의 콜론 뒤의 문자열

In [50]:
brl = 1/2.43
print(brl)
print(format(brl, '0.4f')) # 0,4f 가 format specifier(포맷 명시자))
print('1 BRL = {rate:.2f} USD'.format(rate=brl))

0.4115226337448559
0.4115
1 BRL = 0.41 USD


In [55]:
print(format(42, 'b'))
print(format(2/3, '.1%'))
from datetime import datetime
now = datetime.now()
format(now, '%H:%M:%S')
print("It's now {:%I:%M %p}".format(now))

101010
66.7%
It's now 05:56 PM
